# 02wk-1: Overview (2)

최규빈  
2022-09-08

> 이미지자료분석(2) 데이터정리 $\to$ lrnr $\to$ lrnr.학습() $\to$
> lrnr.예측()

# 강의영상

> <https://youtube.com/playlist?list=PLQqh36zP38-zcq1v38u87lMmD47ujarOZ>

# imports

In [1]:
from fastai.vision.all import * 

# 지난시간 복습

`(1)` 데이터의 정리

In [17]:
path = untar_data(URLs.PETS)/'images'

In [3]:
fnames = get_image_files(path)

In [19]:
f = lambda fname: 'cat' if fname[0].isupper() else 'dog'

In [24]:
dls = ImageDataLoaders.from_name_func(
    path, 
    fnames,
    f, # f대신 (lambda fname: 'cat' if fname[0].isupper() else 'dog') 를 넣어도 가능
    item_tfms=Resize(224))

`(2)` lrnr 오브젝트 생성

In [22]:
lrnr = cnn_learner(dls,resnet34,metrics=error_rate)

`(3)` lrnr.학습()

In [23]:
lrnr.fine_tune(1)

-   fine_tune()은 모든 가중치를 학습하는 것이 아니라 일부만 학습하는
    것임.
-   fine_tune()이외이 방법으로 학습할 수도 있음.

`(4)` lrnr.예측()

(방법1) lrnr.predict() 함수를 이용

In [25]:
lrnr.predict('2022-09-06-hani03.jpg') # 방법1-1
#lrnr.predict(PILImage.create('2022-09-06-hani03.jpg')) # 방법1-2
#lrnr.predict(path.ls()[0]) # 방법1-3

(방법2) lrnr.model(X) 를 이용: X의 shape이 (?,3,224,224)의 형태의
텐서이어야함

In [26]:
X,y = dls.one_batch() # 방법2
lrnr.model(X[0:1]) 

# 프로그래밍 과정 overview

`-` overview

1.  `dls` 오브젝트 생성

2.  `lrnr` 오브젝트 생성

3.  `lrnr.학습()`

4.  `lrnr.예측()`

`-` 비교

|       |    회귀분석(R)    |          이미지분석(CNN)          |         추천시스템          |
|:----------------:|:----------------:|:----------------:|:----------------:|
| 1단계 |   data.frame()    | ImageDataLoaders.from_name_func() | CollabDataLoaders.from_df() |
| 2단계 |       None        |           cnn_learner()           |      collab_learner()       |
| 3단계 |  lm(y~x1+x2,df)   |         lrnr.fine_tune(1)         |         lrnr.fit()          |
| 4단계 | predict(ob,newdf) |   lrnr.predict(), lrnr.model(X)   |        lrnr.model(X)        |

# 숙제

아래의 함수들이 정의된 위치를 찾아보고 경로를 제출하라.

-   ImageDataLoaders.from_name_func
-   cnn_learner
-   lrnr.fine_tune
-   lrnr.predict

단, 여기에서 lrnr는 cnn_learner()로부터 생성된 오브젝트 이다.

**제출예시**

ImageDataLoaders.from_name_func

-   ~/anaconda3/envs/py37/lib/python3.7/site-packages/fastai/vision/data.py